<a href="https://colab.research.google.com/github/fmdazhar/cartesian_6dof/blob/main/examples/ur5e_planning_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UR5e PRM Planning — Step‑by‑Step Notebook

This notebook walks through:

1. Environment and dependencies
2. Loading the UR5e MuJoCo scene
3. Setting up tasks, limits, and the PRM planner
4. Solving IK for a feasible goal configuration
5. Planning in joint space with **PRM/RRT**
6. Executing the trajectory in MuJoCo **offscreen**
7. Recording a **video** of the planned motion

> **Note:** This notebook uses the same concepts as your script (IK + PRM + execution).  
> It renders frames with MuJoCo's offscreen renderer and stitches them into an MP4 for easy playback.


## 1) Dependencies & environment

- You need a working installation of **MuJoCo 3.x** and GPU drivers for offscreen EGL (or fall back to OSMesa).
- Your `cartesian_6dof` package/repo must be importable (pip installable or added to `sys.path`).
- The scene assets from your project should be available, in particular:  
  `universal_robots_ur5e/scene_gripper.xml`

If you need to install packages in this environment, uncomment the `pip` cell below.


In [1]:
# Clone the repository
!git clone https://github.com/fmdazhar/cartesian_6dof.git

# Install the package in editable mode so local changes are picked up
!pip install -e cartesian_6dof

# Change directory to examples
%cd cartesian_6dof/examples

fatal: destination path 'cartesian_6dof' already exists and is not an empty directory.
Obtaining file:///content/cartesian_6dof
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for cartesian_6dof (pyproject.toml) ... done
  Created wheel for cartesian_6dof: filename=cartesian_6dof-0.0.11-py3-none-any.whl size=7090 sha256=68bc85c63cfbe8d7934ffe20b4b26073272784830e1db00b29e67deb53b4102a
  Stored in directory: /tmp/pip-ephem-wheel-cache-dgc9qset/wheels/ea/bf/1c/f226e51cd929014e0a7e015eb7c2e403af0e8a91fe6c01cfde
Successfully built cartesian_6dof
  Attempting uninstall: cartesian_6dof
    Found existing installation: cartesian_6dof 0.0.11
    Uninstalling cartesian_6dof-0.0.11:
      Successfully uninstalled cartesian_6dof-0.0.11
/content/cartesian_6dof/examples


## 2) Configure paths & offscreen rendering

Set where your assets live and enable offscreen rendering.  
If you're on a headless machine with a supported GPU, **EGL** is the fastest option; if it fails, try **osmesa**.


In [2]:
from pathlib import Path
import os

# ---- Update this to the directory that contains your 'universal_robots_ur5e/scene_gripper.xml'
ASSET_ROOT = Path.cwd()

XML_PATH = ASSET_ROOT / "universal_robots_ur5e" / "scene_gripper.xml"
assert XML_PATH.exists(), f"Could not find XML at: {XML_PATH}"

# Prefer EGL for offscreen rendering; fall back to osmesa.
os.environ.setdefault("MUJOCO_GL", "egl")
print("MUJOCO_GL =", os.environ["MUJOCO_GL"])

MUJOCO_GL = egl


## 3) Imports

In [3]:
import numpy as np
import imageio
import time
import cartesian_6dof

import mujoco
from loop_rate_limiters import RateLimiter

from cartesian_6dof.contrib import TeleopMocap  # not used directly here, but kept for parity
from cartesian_6dof.planning.prm import PRMPlanner, PRMPlannerOptions
from cartesian_6dof.planning.path_shortcutting import shortcut_path
from cartesian_6dof.limits.utils import discretize_joint_space_path, construct_geom_id_pairs
from cartesian_6dof.planning.rrt import RRTPlanner, RRTPlannerOptions

print('MuJoCo version:', mujoco.__version__)

MuJoCo version: 3.3.5


## 4) Load model & data

We load the UR5e scene and prepare a `mujoco.MjModel` and `mujoco.MjData`.


In [4]:
model = mujoco.MjModel.from_xml_path(XML_PATH.as_posix())
data = mujoco.MjData(model)

# Reset to a known keyframe if present (e.g., 'home'), otherwise just forward once.
try:
    mujoco.mj_resetDataKeyframe(model, data, model.key('home').id)
except Exception:
    pass
mujoco.mj_forward(model, data)

print('nq (DoF):', model.nq, '| nu (actuators):', model.nu)

nq (DoF): 14 | nu (actuators): 7


## 5) Configure IK tasks & limits

We mirror your script: an end‑effector task for the `pinch` site and a low‑weight posture task.  
We also define velocity limits and collision pairs.


In [5]:
configuration = cartesian_6dof.Configuration(model)

tasks = [
        end_effector_task := cartesian_6dof.FrameTask(
            frame_name="pinch",
            frame_type="site",
            position_cost=1.0,
            orientation_cost=1.0,
            lm_damping=1e-6,
        ),
        posture_task := cartesian_6dof.PostureTask(model, cost=1e-3),
    ]

_arm_collision_geom_names = [
    'shoulder_col',
    'upper_arm_col0',
    'upper_arm_col1',
    'forearm_col0',
    'forearm_col1',
    'wrist_1_col',
    'wrist_2_col',
    'wrist_3_col',
]
arm_collision_geoms = [model.geom(name).id for name in _arm_collision_geom_names]

gripper_proxy_geom_names = ['gripper_dist_box', 'right_pad_ell', 'left_pad_ell']
gripper_proxy_geoms = [model.geom(name).id for name in gripper_proxy_geom_names]

collision_pairs = [
    (arm_collision_geoms + gripper_proxy_geoms, ['wall']),
]
geom_id_pairs = construct_geom_id_pairs(model, collision_pairs, enforce_contype_conaffinity=False)

limits = [
    cartesian_6dof.ConfigurationLimit(model=configuration.model),
]

max_velocities = {
    'shoulder_pan': np.pi,
    'shoulder_lift': np.pi,
    'elbow': np.pi,
    'wrist_1': np.pi,
    'wrist_2': np.pi,
    'wrist_3': np.pi,
}
velocity_limit = cartesian_6dof.VelocityLimit(model, max_velocities)
limits.append(velocity_limit)

# IK settings
SOLVER = 'daqp'
POS_THRESH = 1e-4
ORI_THRESH = 1e-4
MAX_ITERS = 500

print('Configured tasks and limits.')

Configured tasks and limits.


## 6) Build or load a PRM

We either load a saved roadmap or construct one (which can take a while).

In [6]:


# Default RRT options. Adjust as needed.
_RRT_OPTIONS = RRTPlannerOptions(
    max_step_size=0.2,
    max_connection_dist=1.5,
    rrt_connect=True,
    bidirectional_rrt=True,
    rrt_star=True,
    max_rewire_dist=3.0,
    max_planning_time=5.0,
    rng_seed=None,
    fast_return=True,
    goal_biasing_probability=0.3,
    minimum_distance_from_collisions=0.04,
    collision_detection_distance=0.2,
)
planner = RRTPlanner(model=model,
                                  collision_pairs=collision_pairs,
                                  options=_RRT_OPTIONS)



**Uncomment this section to use PRM instead of RRT**


In [7]:
# PRM_FILE = 'my_prm3.graph'  # change if you want a different name
# LOAD_PRM = Path(PRM_FILE).exists()

# if LOAD_PRM:
#     print(f'Loading pre-existing PRM from: {PRM_FILE}')

# options = PRMPlannerOptions(
#     max_step_size=0.05,
#     max_neighbor_radius=3.14,
#     max_neighbor_connections=20,
#     max_construction_nodes=10000,
#     construction_timeout=200.0,
#     rng_seed=None,
#     prm_star=False,
#     prm_file=PRM_FILE if LOAD_PRM else None,
#     minimum_distance_from_collisions=0.05,
#     collision_detection_distance=0.3,
# )

# planner = PRMPlanner(
#     model=model,
#     collision_pairs=collision_pairs,
#     options=options
# )

# if not LOAD_PRM:
#     print(f'Initializing the PRM, this may take up to {options.construction_timeout} seconds...')
#     planner.construct_roadmap()
#     if PRM_FILE:
#         print(f'Saving generated PRM to {PRM_FILE}')
#         planner.graph.save_to_file(PRM_FILE)
# else:
#     print('PRM ready.')

## 7) Solve IK to a target pose

We generate a reachable target pose near the current end effector and run iterative IK to obtain a feasible joint configuration `goal_q`.


In [8]:
def ik_solve_to_target(target_T_wt):
    # Create a fresh configuration to solve IK (without disturbing 'configuration' state)
    ik_cfg = cartesian_6dof.Configuration(model)
    ik_cfg.update(configuration.q.copy())

    # End-effector task target
    end_effector_task.set_target(target_T_wt)
    posture_task.set_target_from_configuration(ik_cfg)

    for _ in range(MAX_ITERS):
        v = cartesian_6dof.solve_ik(ik_cfg, tasks, rate.dt, SOLVER, limits=limits)
        ik_cfg.integrate_inplace(v, rate.dt)
        err = end_effector_task.compute_error(ik_cfg)
        if np.linalg.norm(err[:3]) <= POS_THRESH and np.linalg.norm(err[3:]) <= ORI_THRESH:
            break
    return ik_cfg.q.copy()

# Construct a nearby target (translate + rotate a bit)
target_T = cartesian_6dof.SE3.from_rotation_and_translation(
    cartesian_6dof.SO3(np.array([-0.01589,  0.99964, -0.      , -0.02157], dtype=np.float64)),
    np.array([0.5, 0.35, 0.12073], dtype=np.float64)  # (x, y, z) in meters
)
# Reset sim to the start
data.qpos[:] = 0
data.qvel[:] = 0
data.ctrl[:] = 0
mujoco.mj_resetDataKeyframe(model, data, model.key("home").id)
configuration.update(data.qpos)
mujoco.mj_forward(model, data)

cartesian_6dof.move_mocap_to_frame(model, data, "target", "pinch", "site")
rate = RateLimiter(frequency=200.0, warn=False)

goal_q = ik_solve_to_target(target_T)
start_q = data.qpos.copy()

print('Start q:', start_q[:6])
print('Goal  q:', goal_q[:6])

Start q: [-2.20570036 -1.37522352  1.93687489 -2.07885842 -1.57079982 -0.63489964]
Goal  q: [-1.20037276 -1.19690245  1.69300392 -2.03816584 -1.52555462  0.36977249]


## 8) Plan in joint space with PRM/RRT

In [12]:
path = planner.plan(start_q, goal_q)
if path is None or len(path) <= 1:
    raise RuntimeError('Planner failed to find a path.')

print(f'Found path with {len(path)} waypoints')

# # Optional shortcutting
# short_path = shortcut_path(model, path, geom_id_pairs, max_iters=200, max_step_size=0.05)
# path = short_path
# print(f'Shortcutted path to {len(path)} waypoints')

# Discretize for smooth playback
active_path = discretize_joint_space_path(path, 7e-3)
print(f'Discretized path to {len(active_path)} waypoints')

Found path with 3 waypoints
Discretized path to 219 waypoints


## 9) Execute & record offscreen video

We drive the arm by setting `data.ctrl[:6]` to each waypoint and step the simulator.  
At each step, we render a frame offscreen and append it to an MP4 file.


In [13]:
def make_renderer(model, width=640, height=480):
    # MuJoCo offscreen framebuffer size comes from the XML: <visual><global offwidth/ offheight>
    offw = model.vis.global_.offwidth
    offh = model.vis.global_.offheight

    # Clamp to available framebuffer to avoid ValueError
    w = min(width, offw if offw > 0 else width)
    h = min(height, offh if offh > 0 else height)

    if (w, h) != (width, height):
        print(
            f"[Renderer] Requested {width}x{height}, but model offscreen is {offw}x{offh}. "
            f"Using {w}x{h}. To increase, set <visual><global offwidth=\"{width}\" offheight=\"{height}\"/></visual> in the XML."
        )

    renderer = mujoco.Renderer(model, h, w)

    cam = mujoco.MjvCamera()
    mujoco.mjv_defaultFreeCamera(model, cam)
    return renderer, cam


def render_frame(renderer, data, cam=None):
    if cam is None or cam.fixedcamid == -1:
        renderer.update_scene(data, camera=cam)
    else:
        renderer.update_scene(data, camera=cam)
    return renderer.render()

renderer, cam = make_renderer(model, width=1024, height=768)

video_path = Path('plan_demo.mp4')
writer = imageio.get_writer(video_path.as_posix(), fps=60, codec='libx264')

# Reset sim to the start
data.qpos[:] = 0
data.qvel[:] = 0
data.ctrl[:] = 0
mujoco.mj_resetDataKeyframe(model, data, model.key("home").id)
configuration.update(data.qpos)
mujoco.mj_forward(model, data)
cartesian_6dof.move_mocap_to_frame(model, data, "target", "pinch", "site")

frames = 0
for q_target in active_path:
    data.ctrl[:6] = q_target[:6]

    mujoco.mj_step(model, data)
    cartesian_6dof.move_mocap_to_frame(model, data, "target", "pinch", "site")

    frame = render_frame(renderer, data, cam)
    writer.append_data(frame)
    frames += 1

# --- hold final pose for 100 extra frames ---
for _ in range(50):
    mujoco.mj_step(model, data)
    cartesian_6dof.move_mocap_to_frame(model, data, "target", "pinch", "site")

    frame = render_frame(renderer, data, cam)
    writer.append_data(frame)
    frames += 1
# -------------------------------------------

writer.close()
print(f'Wrote {frames} frames to {video_path.resolve()}')

Wrote 269 frames to /content/cartesian_6dof/examples/plan_demo.mp4


## 10) Play the video inline

In [14]:
from IPython.display import Video, display
display(Video('plan_demo.mp4', embed=True, width=640))